In [ ]:
# Predictions y Scoring - Modelo de Fuga Colsubsidio
# ===================================================
# 
# Objetivo: Generar scoring final y segmentación de riesgo
# - Aplicar modelo final a dataset de test
# - Crear segmentación de clientes por nivel de riesgo
# - Análisis de distribución de scores
# - Preparar datos para recomendaciones de negocio

# %% [markdown]
"""
## 1. Configuración y Carga del Modelo Final
"""

# %%
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys
from pathlib import Path
import joblib

# Importar módulos del proyecto
sys.path.append('..')
from src.business_logic import BusinessLogic

warnings.filterwarnings('ignore')

print("Librerías cargadas correctamente")
print(f"Scoring y segmentación iniciado: {pd.Timestamp.now()}")

# %%
# Cargar modelo entrenado y componentes
data_dir = Path("../data/outputs")

# Verificar archivos necesarios
required_files = ["best_model.pkl", "scaler.pkl", "encoders.pkl"]
missing_files = [f for f in required_files if not (data_dir / f).exists()]

if missing_files:
    print(f"Error: Archivos faltantes: {missing_files}")
    print("Ejecutar primero notebook 04_model_training.ipynb")
    sys.exit()

# Cargar componentes
try:
    model = joblib.load(data_dir / "best_model.pkl")
    scaler = joblib.load(data_dir / "scaler.pkl")
    encoders = joblib.load(data_dir / "encoders.pkl")
    print("Modelo y componentes cargados exitosamente")
except Exception as e:
    print(f"Error cargando componentes: {e}")
    sys.exit()

# Verificar tipo de modelo
print(f"Tipo de modelo: {type(model).__name__}")
if hasattr(model, 'n_estimators'):
    print(f"Configuración: {model.n_estimators} estimadores")

# %%
# Cargar datos de test preparados
test_data_path = Path("../data/processed/test_with_features.csv")

if not test_data_path.exists():
    print("Error: Datos de test con features no encontrados")
    print("Ejecutar notebooks 02 y 03 primero")
    sys.exit()

test_data = pd.read_csv(test_data_path)

print("Datos de test cargados:")
print(f"  Registros: {len(test_data):,}")
print(f"  Columnas: {len(test_data.columns)}")

# Verificar estructura
if 'id' not in test_data.columns:
    print("Advertencia: Columna 'id' no encontrada")
    test_data['id'] = range(len(test_data))

print(f"  IDs únicos: {test_data['id'].nunique():,}")

# %% [markdown]
"""
## 2. Preparación de Datos para Scoring
"""

# %%
# Preparar datos para scoring
def prepare_test_data_for_scoring(test_df):
    """Prepara datos de test aplicando las mismas transformaciones que en entrenamiento."""
    
    print("Preparando datos de test para scoring...")
    
    # Separar ID y features
    test_ids = test_df['id'] if 'id' in test_df.columns else range(len(test_df))
    X_test = test_df.drop(['id'], axis=1, errors='ignore')
    
    print(f"Features para scoring: {X_test.shape}")
    
    # Identificar variables categóricas
    categorical_vars = X_test.select_dtypes(include=['object']).columns.tolist()
    print(f"Variables categóricas detectadas: {len(categorical_vars)}")
    
    # Aplicar encoding a variables categóricas
    X_test_encoded = X_test.copy()
    
    for col in categorical_vars:
        if col in encoders:
            print(f"  Codificando: {col}")
            
            # Convertir a string y llenar nulos
            X_test_encoded[col] = X_test_encoded[col].astype(str).fillna('Unknown')
            
            # Aplicar encoder entrenado
            encoder = encoders[col]
            known_values = set(encoder.classes_)
            
            def safe_transform(value):
                if value in known_values:
                    return encoder.transform([value])[0]
                else:
                    return -1  # Valor para categorías no vistas
            
            X_test_encoded[col] = X_test_encoded[col].apply(safe_transform)
            
            # Reportar valores nuevos
            unknown_count = (X_test_encoded[col] == -1).sum()
            if unknown_count > 0:
                print(f"    Valores no vistos: {unknown_count}")
        else:
            print(f"  Advertencia: Encoder no encontrado para {col}")
    
    # Aplicar escalado
    print("Aplicando escalado...")
    X_test_scaled = scaler.transform(X_test_encoded)
    
    print(f"Datos preparados: {X_test_scaled.shape}")
    
    return X_test_scaled, test_ids, X_test_encoded.columns.tolist()

X_test_scaled, test_ids, feature_names = prepare_test_data_for_scoring(test_data)

# %% [markdown]
"""
## 3. Generación de Scores de Fuga
"""

# %%
# Generar predicciones del modelo
def generate_churn_scores():
    """Genera scores de probabilidad de fuga."""
    
    print("Generando scores de fuga...")
    
    # Predicciones de probabilidad
    churn_probabilities = model.predict_proba(X_test_scaled)[:, 1]
    
    # Predicciones binarias (usando umbral 0.5)
    churn_predictions = model.predict(X_test_scaled)
    
    print(f"Scores generados:")
    print(f"  Total clientes: {len(churn_probabilities):,}")
    print(f"  Rango probabilidades: {churn_probabilities.min():.4f} - {churn_probabilities.max():.4f}")
    print(f"  Media: {churn_probabilities.mean():.4f}")
    print(f"  Mediana: {np.median(churn_probabilities):.4f}")
    
    # Distribución de predicciones binarias
    unique_preds, counts = np.unique(churn_predictions, return_counts=True)
    print(f"\nDistribución de predicciones binarias:")
    for pred, count in zip(unique_preds, counts):
        label = "No Fuga" if pred == 0 else "Fuga"
        pct = count / len(churn_predictions) * 100
        print(f"  {label}: {count:,} ({pct:.1f}%)")
    
    return churn_probabilities, churn_predictions

churn_probabilities, churn_predictions = generate_churn_scores()

# %%
# Análisis de distribución de scores
def analyze_score_distribution():
    """Analiza la distribución de scores de fuga."""
    
    print("Analizando distribución de scores...")
    
    # Estadísticas descriptivas
    percentiles = np.percentile(churn_probabilities, [5, 10, 25, 50, 75, 90, 95, 99])
    
    print(f"\nEstadísticas de distribución:")
    print(f"  P5:  {percentiles[0]:.4f}")
    print(f"  P10: {percentiles[1]:.4f}")
    print(f"  P25: {percentiles[2]:.4f}")
    print(f"  P50: {percentiles[3]:.4f}")
    print(f"  P75: {percentiles[4]:.4f}")
    print(f"  P90: {percentiles[5]:.4f}")
    print(f"  P95: {percentiles[6]:.4f}")
    print(f"  P99: {percentiles[7]:.4f}")
    
    # Visualización de distribución
    fig_dist = make_subplots(
        rows=1, cols=2,
        subplot_titles=['Histograma de Scores', 'Box Plot de Distribución'],
        specs=[[{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Histograma
    fig_dist.add_trace(
        go.Histogram(
            x=churn_probabilities,
            nbinsx=50,
            name='Distribución de Scores',
            marker_color='lightblue'
        ),
        row=1, col=1
    )
    
    # Box plot
    fig_dist.add_trace(
        go.Box(
            y=churn_probabilities,
            name='Distribución',
            marker_color='lightcoral',
            boxpoints='outliers'
        ),
        row=1, col=2
    )
    
    fig_dist.update_layout(
        title_text="Distribución de Scores de Fuga",
        height=500,
        showlegend=False
    )
    
    fig_dist.update_xaxes(title_text="Probabilidad de Fuga", row=1, col=1)
    fig_dist.update_yaxes(title_text="Frecuencia", row=1, col=1)
    fig_dist.update_yaxes(title_text="Probabilidad de Fuga", row=1, col=2)
    
    fig_dist.show()
    
    return percentiles

score_percentiles = analyze_score_distribution()

# %% [markdown]
"""
## 4. Segmentación de Riesgo
"""

# %%
# Implementar segmentación de riesgo
def create_risk_segmentation():
    """Crea segmentación de clientes por nivel de riesgo."""
    
    print("Creando segmentación de riesgo...")
    
    # Inicializar business logic
    business_logic = BusinessLogic()
    
    # Crear segmentación usando percentiles
    risk_segments, thresholds = business_logic.create_risk_segments(churn_probabilities)
    
    print(f"Umbrales de segmentación:")
    print(f"  Alto Riesgo (top 5%): >= {thresholds['high_risk']:.4f}")
    print(f"  Medio-Alto (top 20%): >= {thresholds['medium_high']:.4f}")
    print(f"  Medio (top 40%): >= {thresholds['medium']:.4f}")
    print(f"  Bajo Riesgo: < {thresholds['medium']:.4f}")
    
    # Contar clientes por segmento
    segment_counts = pd.Series(risk_segments).value_counts()
    segment_props = pd.Series(risk_segments).value_counts(normalize=True) * 100
    
    print(f"\nDistribución por segmento:")
    for segment in ['Alto_Riesgo', 'Medio_Alto_Riesgo', 'Medio_Riesgo', 'Bajo_Riesgo']:
        if segment in segment_counts.index:
            count = segment_counts[segment]
            prop = segment_props[segment]
            print(f"  {segment}: {count:,} clientes ({prop:.1f}%)")
    
    # Visualización de segmentación
    fig_segments = go.Figure(data=[
        go.Pie(
            labels=segment_counts.index,
            values=segment_counts.values,
            hole=0.4,
            marker_colors=['#FF6B6B', '#FFB347', '#87CEEB', '#98FB98']
        )
    ])
    
    fig_segments.update_layout(
        title="Segmentación de Clientes por Riesgo de Fuga",
        annotations=[dict(text=f'Total<br>{len(risk_segments):,}<br>Clientes', 
                         x=0.5, y=0.5, font_size=16, showarrow=False)],
        height=500
    )
    
    fig_segments.show()
    
    return risk_segments, thresholds, segment_counts

risk_segments, risk_thresholds, segment_counts = create_risk_segmentation()

# %%
# Análisis detallado por segmento
def analyze_segments_detail():
    """Analiza características de cada segmento de riesgo."""
    
    print("Análisis detallado por segmento...")
    
    # Crear DataFrame con resultados
    results_df = pd.DataFrame({
        'id': test_ids,
        'churn_probability': churn_probabilities,
        'churn_prediction': churn_predictions,
        'risk_segment': risk_segments
    })
    
    # Calcular percentil de riesgo
    results_df['risk_percentile'] = results_df['churn_probability'].rank(pct=True) * 100
    
    print(f"\nAnálisis por segmento de riesgo:")
    
    for segment in ['Alto_Riesgo', 'Medio_Alto_Riesgo', 'Medio_Riesgo', 'Bajo_Riesgo']:
        if segment in results_df['risk_segment'].values:
            segment_data = results_df[results_df['risk_segment'] == segment]
            
            print(f"\n{segment.replace('_', ' ').upper()}:")
            print(f"  Clientes: {len(segment_data):,}")
            print(f"  Score promedio: {segment_data['churn_probability'].mean():.4f}")
            print(f"  Score mínimo: {segment_data['churn_probability'].min():.4f}")
            print(f"  Score máximo: {segment_data['churn_probability'].max():.4f}")
            print(f"  Percentil promedio: {segment_data['risk_percentile'].mean():.1f}")
    
    return results_df

results_df = analyze_segments_detail()

# %%
# Análisis de scores por segmento
def visualize_segments_distribution():
    """Visualiza la distribución de scores por segmento."""
    
    print("Visualizando distribución por segmento...")
    
    # Box plot por segmento
    fig_box = px.box(
        results_df,
        x='risk_segment',
        y='churn_probability',
        title='Distribución de Scores por Segmento de Riesgo',
        labels={'churn_probability': 'Probabilidad de Fuga', 'risk_segment': 'Segmento de Riesgo'},
        color='risk_segment',
        color_discrete_map={
            'Alto_Riesgo': '#FF6B6B',
            'Medio_Alto_Riesgo': '#FFB347', 
            'Medio_Riesgo': '#87CEEB',
            'Bajo_Riesgo': '#98FB98'
        }
    )
    
    fig_box.update_layout(height=500, showlegend=False)
    fig_box.show()
    
    # Histograma apilado
    fig_hist = go.Figure()
    
    colors = ['#FF6B6B', '#FFB347', '#87CEEB', '#98FB98']
    segments = ['Alto_Riesgo', 'Medio_Alto_Riesgo', 'Medio_Riesgo', 'Bajo_Riesgo']
    
    for i, segment in enumerate(segments):
        if segment in results_df['risk_segment'].values:
            segment_scores = results_df[results_df['risk_segment'] == segment]['churn_probability']
            
            fig_hist.add_trace(go.Histogram(
                x=segment_scores,
                name=segment.replace('_', ' '),
                opacity=0.7,
                marker_color=colors[i],
                nbinsx=30
            ))
    
    fig_hist.update_layout(
        title='Distribución de Scores por Segmento (Histograma)',
        xaxis_title='Probabilidad de Fuga',
        yaxis_title='Frecuencia',
        barmode='overlay',
        height=500
    )
    
    fig_hist.show()

visualize_segments_distribution()

# %% [markdown]
"""
## 5. Validación de Segmentación
"""

# %%
# Validar calidad de la segmentación
def validate_segmentation():
    """Valida la calidad de la segmentación creada."""
    
    print("Validando calidad de segmentación...")
    
    # 1. Verificar separación entre segmentos
    print(f"\n1. Separación entre segmentos:")
    segment_stats = results_df.groupby('risk_segment')['churn_probability'].agg(['min', 'max', 'mean']).round(4)
    
    for segment in segment_stats.index:
        stats = segment_stats.loc[segment]
        print(f"  {segment}: rango {stats['min']:.4f} - {stats['max']:.4f}, promedio {stats['mean']:.4f}")
    
    # 2. Verificar no solapamiento (excepto en bordes)
    print(f"\n2. Verificación de solapamiento:")
    segments_ordered = ['Bajo_Riesgo', 'Medio_Riesgo', 'Medio_Alto_Riesgo', 'Alto_Riesgo']
    
    for i in range(len(segments_ordered) - 1):
        current_seg = segments_ordered[i]
        next_seg = segments_ordered[i + 1]
        
        if current_seg in segment_stats.index and next_seg in segment_stats.index:
            current_max = segment_stats.loc[current_seg, 'max']
            next_min = segment_stats.loc[next_seg, 'min']
            
            if current_max <= next_min:
                print(f"  {current_seg} vs {next_seg}: Correcta separación")
            else:
                overlap = current_max - next_min
                print(f"  {current_seg} vs {next_seg}: Solapamiento de {overlap:.4f}")
    
    # 3. Verificar distribución de tamaños
    print(f"\n3. Distribución de tamaños:")
    total_clients = len(results_df)
    
    for segment in segment_counts.index:
        count = segment_counts[segment]
        expected_ranges = {
            'Alto_Riesgo': (0.03, 0.07),  # 3-7%
            'Medio_Alto_Riesgo': (0.12, 0.18),  # 12-18%
            'Medio_Riesgo': (0.18, 0.22),  # 18-22%
            'Bajo_Riesgo': (0.55, 0.65)   # 55-65%
        }
        
        actual_prop = count / total_clients
        expected_range = expected_ranges.get(segment, (0, 1))
        
        if expected_range[0] <= actual_prop <= expected_range[1]:
            status = "OK"
        else:
            status = "Revisar"
        
        print(f"  {segment}: {actual_prop:.1%} ({status})")

validate_segmentation()

# %% [markdown]
"""
## 6. Análisis Complementario con Datos Demográficos
"""

# %%
# Análisis con variables demográficas
def analyze_demographics_by_segment():
    """Analiza características demográficas por segmento."""
    
    print("Análisis demográfico por segmento...")
    
    # Intentar cargar datos demográficos adicionales del test
    demographic_vars = ['segmento', 'edad', 'estrato', 'benefits_index']
    available_demo_vars = [var for var in demographic_vars if var in test_data.columns]
    
    if available_demo_vars:
        print(f"Variables demográficas disponibles: {available_demo_vars}")
        
        # Agregar variables demográficas al DataFrame de resultados
        enhanced_results = results_df.copy()
        
        for var in available_demo_vars:
            enhanced_results[var] = test_data[var].values
        
        # Análisis por segmento
        for segment in ['Alto_Riesgo', 'Medio_Alto_Riesgo']:
            if segment in enhanced_results['risk_segment'].values:
                segment_data = enhanced_results[enhanced_results['risk_segment'] == segment]
                
                print(f"\n{segment.replace('_', ' ').upper()}:")
                print(f"  Tamaño: {len(segment_data):,} clientes")
                
                # Análisis de variables disponibles
                for var in available_demo_vars:
                    if var == 'edad':
                        avg_age = segment_data[var].mean()
                        print(f"  Edad promedio: {avg_age:.1f} años")
                    elif var == 'segmento':
                        top_segment = segment_data[var].mode().iloc[0] if len(segment_data) > 0 else 'N/A'
                        print(f"  Segmento más común: {top_segment}")
                    elif var == 'benefits_index':
                        avg_benefits = segment_data[var].mean()
                        print(f"  Índice de beneficios promedio: {avg_benefits:.2f}")
                    elif var == 'estrato':
                        avg_estrato = segment_data[var].mean()
                        print(f"  Estrato promedio: {avg_estrato:.1f}")
    else:
        print("No hay variables demográficas disponibles para análisis adicional")

analyze_demographics_by_segment()

# %% [markdown]
"""
## 7. Exportación de Resultados de Scoring
"""

# %%
# Exportar resultados finales
def export_scoring_results():
    """Exporta todos los resultados del scoring."""
    
    print("Exportando resultados de scoring...")
    
    output_dir = Path("../data/outputs")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 1. Exportar DataFrame principal con scores y segmentación
    main_results_path = output_dir / "client_risk_segmentation.csv"
    results_df.to_csv(main_results_path, index=False)
    print(f"Segmentación principal guardada: {main_results_path}")
    
    # 2. Crear resumen por segmento
    segment_summary = []
    for segment in segment_counts.index:
        segment_data = results_df[results_df['risk_segment'] == segment]
        
        segment_summary.append({
            'segment': segment,
            'client_count': len(segment_data),
            'percentage': len(segment_data) / len(results_df) * 100,
            'avg_probability': segment_data['churn_probability'].mean(),
            'min_probability': segment_data['churn_probability'].min(),
            'max_probability': segment_data['churn_probability'].max(),
            'threshold_used': risk_thresholds.get(segment.lower().replace('_riesgo', '').replace('medio_alto', 'medium_high'), 'N/A')
        })
    
    segment_summary_df = pd.DataFrame(segment_summary)
    summary_path = output_dir / "risk_segmentation_summary.csv"
    segment_summary_df.to_csv(summary_path, index=False)
    print(f"Resumen por segmento guardado: {summary_path}")
    
    # 3. Exportar configuración de umbrales
    thresholds_config = {
        'segmentation_date': pd.Timestamp.now().isoformat(),
        'total_clients': len(results_df),
        'thresholds': risk_thresholds,
        'segment_counts': segment_counts.to_dict()
    }
    
    import json
    config_path = output_dir / "segmentation_config.json"
    with open(config_path, 'w') as f:
        json.dump(thresholds_config, f, indent=2)
    print(f"Configuración guardada: {config_path}")
    
    # 4. Estadísticas de distribución
    distribution_stats = {
        'percentiles': {
            f'P{p}': float(np.percentile(churn_probabilities, p))
            for p in [5, 10, 25, 50, 75, 90, 95, 99]
        },
        'basic_stats': {
            'mean': float(churn_probabilities.mean()),
            'std': float(churn_probabilities.std()),
            'min': float(churn_probabilities.min()),
            'max': float(churn_probabilities.max())
        }
    }
    
    stats_path = output_dir / "score_distribution_stats.json"
    with open(stats_path, 'w') as f:
        json.dump(distribution_stats, f, indent=2)
    print(f"Estadísticas de distribución guardadas: {stats_path}")

export_scoring_results()

# %% [markdown]
"""
## 8. Resumen Ejecutivo de Scoring
"""

# %%
# Resumen ejecutivo final
print("=" * 60)
print("RESUMEN EJECUTIVO - SCORING Y SEGMENTACIÓN")
print("=" * 60)

print(f"\nSCORING COMPLETADO:")
print(f"  Total clientes procesados: {len(results_df):,}")
print(f"  Rango de probabilidades: {churn_probabilities.min():.4f} - {churn_probabilities.max():.4f}")
print(f"  Score promedio: {churn_probabilities.mean():.4f}")

print(f"\nSEGMENTACIÓN DE RIESGO:")
for segment in ['Alto_Riesgo', 'Medio_Alto_Riesgo', 'Medio_Riesgo', 'Bajo_Riesgo']:
    if segment in segment_counts.index:
        count = segment_counts[segment]
        prop = count / len(results_df) * 100
        avg_score = results_df[results_df['risk_segment'] == segment]['churn_probability'].mean()
        print(f"  {segment.replace('_', ' ')}: {count:,} clientes ({prop:.1f}%) - Score promedio: {avg_score:.4f}")

print(f"\nUMBRALES APLICADOS:")
print(f"  Alto Riesgo: >= {risk_thresholds['high_risk']:.4f}")
print(f"  Medio-Alto: >= {risk_thresholds['medium_high']:.4f}")
print(f"  Medio: >= {risk_thresholds['medium']:.4f}")

print(f"\nCLIENTES PRIORITARIOS PARA CAMPAÑA:")
priority_clients = segment_counts.get('Alto_Riesgo', 0) + segment_counts.get('Medio_Alto_Riesgo', 0)
priority_pct = priority_clients / len(results_df) * 100
print(f"  Alto + Medio-Alto Riesgo: {priority_clients:,} clientes ({priority_pct:.1f}%)")
print(f"  Estos clientes requieren atención inmediata")

print(f"\nCALIDAD DE SEGMENTACIÓN:")
print(f"  Separación clara entre segmentos: Verificada")
print(f"  Distribución balanceada: Adecuada")
print(f"  Umbrales basados en percentiles: Consistentes")

print(f"\nARCHIVOS GENERADOS:")
print(f"  - client_risk_segmentation.csv: Dataset completo con scores")
print(f"  - risk_segmentation_summary.csv: Resumen por segmento")
print(f"  - segmentation_config.json: Configuración de umbrales")
print(f"  - score_distribution_stats.json: Estadísticas de distribución")

print(f"\nPRÓXIMOS PASOS:")
print(f"  1. Análisis de business insights por segmento")
print(f"  2. Cálculo de ROI y presupuestos de campaña")
print(f"  3. Recomendaciones específicas de retención")
print(f"  4. Plan de implementación de campañas")

print(f"\n" + "=" * 60)
print(f"SCORING Y SEGMENTACIÓN COMPLETADOS")
print(f"Fecha: {pd.Timestamp.now()}")
print(f"Siguiente paso: Business Insights (Notebook 07)")
print("=" * 60)

# %%